In [1]:
from pyecsca.sca.trace_set import InspectorTraceSet, PickleTraceSet, HDF5TraceSet
from pyecsca.sca.trace import Trace
from pyecsca.ec.params import get_params
from pyecsca.ec.mult import LadderMultiplier
from pyecsca.ec.point import Point
from pyecsca.ec.mod import Mod
from tqdm.auto import tqdm

In [2]:
curve25519 = get_params("other", "Curve25519", "xz")
p = curve25519.curve.prime
n = curve25519.order
ladder = curve25519.curve.coordinate_model.formulas["ladd-1987-m"]
dbl = curve25519.curve.coordinate_model.formulas["dbl-1987-m"]
mult = LadderMultiplier(ladder, dbl, complete=False)

In [3]:
def fix_trace(trace, **kwargs):
    data = trace.meta["data"]
    group_data = bytearray([data[0]])
    
    point_data = bytearray(data[1:33])
    point_data[-1] &= 0b01111111
    
    scalar_data = bytearray(data[33:65])
    # From the impl:
    # https://github.com/sca-secure-library-sca25519/sca25519/blob/main/STM32F407-unprotected/crypto/scalarmult/scalarmult_25519.c#L125
    scalar_data[0] &= 248
    scalar_data[31] &= 127
    scalar_data[31] |= 64
    
    result_data = bytearray(data[65:97])
    result_data[-1] &= 0b01111111
    
    point_x = int.from_bytes(point_data, byteorder="little")
    result_x = int.from_bytes(result_data, byteorder="little")
    scalar = int.from_bytes(scalar_data, byteorder="little")
    #print(data.hex())
    #print(group_data.hex(), point_data.hex(), scalar_data.hex(), result_data.hex())
    assert len(group_data) == 1
    assert len(point_data) == 32
    assert len(scalar_data) == 32
    assert len(result_data) == 32

    point = Point(curve25519.curve.coordinate_model, X=Mod(point_x, p), Z=Mod(1, p))
    result = Point(curve25519.curve.coordinate_model, X=Mod(result_x, p), Z=Mod(1, p))
    point_affs = curve25519.curve.affine_lift_x(Mod(point_x, p))
    #assert point_affs
    print(bool(point_affs), end=", ")
    result_affs = curve25519.curve.affine_lift_x(Mod(result_x, p))
    #assert result_affs
    print(bool(result_affs), end=", ")
    if point_affs and result_affs:
        point_aff = list(point_affs)[0]
        result_aff = list(result_affs)[0]
        r_aff = curve25519.curve.affine_multiply(point_aff, scalar)
        mine_aff = r_aff.x
    else:
        mine_aff = None
    
    mult.init(curve25519, point)
    r = mult.multiply(scalar)
    mine = r.X / r.Z
    their = result.X / result.Z
    print(mine_aff == their, end=", ")
    print(mine == their)
    #assert mine == their

    return Trace(trace.samples, {"scalar": scalar,
                                 "point": point,
                                 "result": result, **kwargs})

In [10]:
unprotected_100 = InspectorTraceSet.read("sca25519_unprotected_100.trs", scale=False)
ephemeral_100 = InspectorTraceSet.read("sca25519_ephemeral_100.trs", scale=False)
static_100 = InspectorTraceSet.read("sca25519_static_100.trs", scale=False)

In [13]:
traces = []
for trace in tqdm(unprotected_100[:10]):
    traces.append(fix_trace(trace, group="unprotected"))
for trace in tqdm(ephemeral_100[:10]):
    traces.append(fix_trace(trace, group="ephemeral"))
for trace in tqdm(static_100[:10]):
    traces.append(fix_trace(trace, group="static"))
# Remove the metadata for these traces, except the group, because some of the data is clearly wrong.
for trace in traces:
    trace.meta = {"group": trace.meta["group"]}

  0%|          | 0/10 [00:00<?, ?it/s]

True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True


  0%|          | 0/10 [00:00<?, ?it/s]

True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
False, False, False, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True


  0%|          | 0/10 [00:00<?, ?it/s]

True, True, False, False
True, True, False, False
True, True, False, False
True, True, False, False
True, True, False, False
True, True, False, False
True, True, False, False
True, True, False, False
True, True, False, False
True, True, False, False


In [14]:
pickle = PickleTraceSet(*traces).write("traces_A_new.pickle")
hdf5 = HDF5TraceSet(*traces).write("traces_A_new.hdf5")

In [4]:
save_940 = InspectorTraceSet.read("sca25519_unprotected_3k+SAVE(940,2000).trs", scale=False)

In [5]:
traces = []
for trace in tqdm(save_940[1000:]):
    fixed = fix_trace(trace)
    traces.append(fixed)

  0%|          | 0/1000 [00:00<?, ?it/s]

True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True, True, True
True, True,

In [6]:
pickle = PickleTraceSet(*traces).write("traces_B_new.pickle")
hdf5 = HDF5TraceSet(*traces).write("traces_B_new.hdf5")

In [5]:
traces = []
for trace in tqdm(save_940[:1000]):
    fixed = fix_trace(trace)
    traces.append(fixed)

  0%|          | 0/1000 [00:00<?, ?it/s]

True, True, True, True
True, True, True, True
False, False, False, True
False, False, False, True
True, True, True, True
False, False, False, True
True, True, True, True
False, False, False, True
False, False, False, True
False, False, False, True
True, True, True, True
True, True, True, True
False, False, False, True
True, True, True, True
True, True, True, True
False, False, False, True
False, False, False, True
True, True, True, True
True, True, True, True
True, True, True, True
False, False, False, True
True, True, True, True
False, False, False, True
True, True, True, True
False, False, False, True
True, True, True, True
True, True, True, True
False, False, False, True
False, False, False, True
True, True, True, True
False, False, False, True
True, True, True, True
False, False, False, True
True, True, True, True
False, False, False, True
True, True, True, True
True, True, True, True
False, False, False, True
False, False, False, True
False, False, False, True
True, True, True, Tr

In [6]:
pickle = PickleTraceSet(*traces).write("traces_D_new.pickle")
hdf5 = HDF5TraceSet(*traces).write("traces_D_new.hdf5")

In [4]:
trim = InspectorTraceSet.read("sca25519_unprotected_3k_trim.trs", scale=False)

In [5]:
traces = []
for trace in tqdm(trim):
    fixed = fix_trace(trace)
    traces.append(fixed)

  0%|          | 0/1000 [00:00<?, ?it/s]

True, True, True, True
True, True, True, True
False, False, False, True
False, False, False, True
True, True, True, True
False, False, False, True
True, True, True, True
False, False, False, True
False, False, False, True
False, False, False, True
True, True, True, True
True, True, True, True
False, False, False, True
True, True, True, True
True, True, True, True
False, False, False, True
False, False, False, True
True, True, True, True
True, True, True, True
True, True, True, True
False, False, False, True
True, True, True, True
False, False, False, True
True, True, True, True
False, False, False, True
True, True, True, True
True, True, True, True
False, False, False, True
False, False, False, True
True, True, True, True
False, False, False, True
True, True, True, True
False, False, False, True
True, True, True, True
False, False, False, True
True, True, True, True
True, True, True, True
False, False, False, True
False, False, False, True
False, False, False, True
True, True, True, Tr

In [6]:
pickle = PickleTraceSet(*traces).write("traces_C_new.pickle")
hdf5 = HDF5TraceSet(*traces).write("traces_C_new.hdf5")